# 一、道德词典

In [2]:
import pandas as pd
import numpy as np
import jieba
import moralstrength
from moralstrength import lexicon_use
from moralstrength.moralstrength import estimate_morals

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import cmfd

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BITTER~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.463 seconds.
Prefix dict has been built successfully.


## （一）英文语料库道德词典分析

In [5]:
#load语料库
df_eng = pd.read_excel(r'E:\[- -] C\文本数据分析\上课实例\text_analysis_twitter_sample.xlsx',index_col = 0)
df_eng.head()

,index,id,screen_name,time,link,text,source
0,49374,890587249372524544,auctnr1,2017-07-27T10:58:41-04:00,https://www.twitter.com/Reuters/statuses/89058...,"RT @Reuters MORE: Top U.S. general says, given...",Twitter for iPhone
1,83246,899354463055618048,SenatorTester,2017-08-20T15:36:27-04:00,https://www.twitter.com/SenatorTester/statuses...,T-minus 2 days until our first-ever Last Best ...,Twitter Web Client
2,100988,903272105738985472,KeithRothfus,2017-08-31T11:03:46-04:00,https://www.twitter.com/KeithRothfus/statuses/...,Please know that help is available. Visit http...,Twitter Web Client
3,193395,921001114409021440,HASCRepublicans,2017-10-19T09:12:31-04:00,https://www.twitter.com/HASCRepublicans/status...,Literally flying the wings off the A-10 in fig...,Twitter Web Client
4,12662,884911451449774080,SteveKnight25,2017-07-11T19:05:05-04:00,https://www.twitter.com/SteveKnight25/statuses...,Today the House unanimously passed my bill #HR...,Twitter Web Client


In [ ]:
#英文内置道德词
lexicon_use.select_version('lateset')

In [9]:
#传入参数：数据表，自动加入道德词典进行分析
df_eng_morals = estimate_morals(df_eng['text'].tolist(),process=True)
df_eng = pd.concat([df_eng,df_eng_morals],axis=1)

D:\anaconda\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [10]:
df_eng.head()

,index,id,screen_name,time,link,text,source,care,fairness,loyalty,authority,purity
0,49374,890587249372524544,auctnr1,2017-07-27T10:58:41-04:00,https://www.twitter.com/Reuters/statuses/89058...,"RT @Reuters MORE: Top U.S. general says, given...",Twitter for iPhone,NaN,NaN,NaN,NaN,NaN
1,83246,899354463055618048,SenatorTester,2017-08-20T15:36:27-04:00,https://www.twitter.com/SenatorTester/statuses...,T-minus 2 days until our first-ever Last Best ...,Twitter Web Client,NaN,NaN,NaN,NaN,NaN
2,100988,903272105738985472,KeithRothfus,2017-08-31T11:03:46-04:00,https://www.twitter.com/KeithRothfus/statuses/...,Please know that help is available. Visit http...,Twitter Web Client,NaN,NaN,NaN,NaN,NaN
3,193395,921001114409021440,HASCRepublicans,2017-10-19T09:12:31-04:00,https://www.twitter.com/HASCRepublicans/status...,Literally flying the wings off the A-10 in fig...,Twitter Web Client,4.0,NaN,NaN,NaN,NaN
4,12662,884911451449774080,SteveKnight25,2017-07-11T19:05:05-04:00,https://www.twitter.com/SteveKnight25/statuses...,Today the House unanimously passed my bill #HR...,Twitter Web Client,NaN,NaN,NaN,NaN,NaN


## （二）中文语料库道德词典分析

In [8]:
#导入中文语料库
df_chn = pd.read_excel(r'E:\[- -] C\文本数据分析\text_analysis_ad.xlsx',index_col = 0)
df_chn.head()

In [11]:
#导入cmfd中文道德词典
chn_moral = pd.read_csv(r'E:\[- -] C\文本数据分析\上课实例\cmfd_civictech.csv')
chn_moral

,chinese,foundation
0,同情,care
1,一臂之力,care
2,一见倾心,care
3,三个代表,care
4,上阵杀敌,care
...,...,...
6133,随和,general
6134,雅正,general
6135,雷打不动,general
6136,马马虎虎,general


In [12]:
moral_dict = chn_moral.groupby('foundation')['chinese'].apply(list).to_dict()

In [13]:
#设置道德词计算公式
def moral_quantity(text):
    
    if isinstance(text, str):
        moral_word_total = 0
        moral_word = {}
        moral_num = {}

        for key in moral_dict.keys():
            moral_word[key] = []
        for word in jieba.cut(text):
            for key in moral_dict.keys():
                if word in moral_dict[key]:
                    moral_word[key].append(word)

        for key in moral_word.keys():
            moral_word_total += len(moral_word[key])
        if moral_word_total == 0:
            return None

        for key in moral_word.keys():
            moral_num[key] = len(moral_word[key]) / moral_word_total

    return moral_num

In [16]:
text_test = """在一个古老的王国里，有一位年轻的王子，名叫艾伦。他爱上了王国中一个普通的女孩，名叫艾丽丝。然而，王子的父亲，即国王，却不同意这段感情，因为他认为艾丽丝的出身不够高贵，不适合成为王室的一员。

王子面临着艰难的选择，他可以违抗父王的意愿和艾丽丝在一起，但这意味着他要放弃王位和王国的责任。另一方面，他也可以遵从父王的意愿，选择一个地位高贵但他并不爱的女子，这样可以继续承担王国的责任。

在经过深思熟虑后，王子决定选择违抗父王的意愿，选择与艾丽丝在一起。他认为爱情比地位和财富更重要，而且他也相信只有在爱情的支持下，他才能成为一个真正的幸福的国王。

尽管他的决定引起了一些争议和反对，但王子坚持了自己的选择，并与艾丽丝结为夫妻。他们的爱情和幸福成为了王国中的典范，人们开始理解和尊重不同阶层之间的爱情，认识到爱情是超越地位和财富的。"""

In [18]:
import cmfd
cmfd.moral_quantity(text_test)

{('altr',): 0.0,
 ('auth',): 0.5161290322580645,
 ('care',): 0.06451612903225806,
 ('dili',): 0.0,
 ('fair',): 0.0,
 ('general',): 0.06451612903225806,
 ('libe',): 0.0,
 ('loya',): 0.12903225806451613,
 ('mode',): 0.03225806451612903,
 ('resi',): 0.03225806451612903,
 ('sanc',): 0.16129032258064516,
 ('wast',): 0.0}

In [14]:
#在语料中运用公式
df_chn['chn_moral'] = df_chn['广告文本'].apply(moral_quantity)
df_chn.head()

,index,商品名称,商品类别,出版年,广告文本,广告标题,chn_moral
0,1596,五华牌香烟[May Blossom],烟草制品,1932,兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟 二十枝装每包售国币大洋二角 五十枝装每罐售国币...,"五华牌香烟,""兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟""",None
1,5627,韦廉士红色清导丸[Dr.Willams' Pink Pills For Pale People],药品,1918,讲求卫生为人生本性天理固然也 人生首贵逐日大便通畅有序为天然所当如此也如若大便不利大肠阻塞则...,韦廉士红色清导丸:'讲求卫生为人生本性天理固然也',"{'altr': 0.0, 'auth': 0.25, 'care': 0.25, 'dil..."
2,13532,大炮台香烟[Three Castles Cigarettes],烟草制品,1935,"香味馥郁,不让名花 另有三炮台出售","大炮台香烟,""香味馥郁 不让名花""",None
3,1133,婴孩自己药片[Baby's Own],药品,1930,差肩儿女 秀慧康强 闽有佳音讃羡婴孩自己药片 每年此际小儿患肠胃病者甚多而尤以南方各地天气翳...,"婴孩自己药片,""差肩儿女 秀慧康强 闽有佳音赞美婴孩自己药片""","{'altr': 0.0, 'auth': 0.2, 'care': 0.2, 'dili'..."
4,3146,亚士北罗药片[Aspro],药品,1933,何以亚士北罗是妇女们的腻友？各国妇女力证亚士北罗药片是她们最需要的药物！为什么？她们的经验知...,"亚士北罗药片,""妇女之腻友""","{'altr': 0.0, 'auth': 0.16666666666666666, 'ca..."


In [15]:
#可视化结果
chn_moral_df = pd.DataFrame(columns=['altr', 'auth', 'care', 'dili', 'fair', 'general', 'libe', 'loya', 'mode', 'resi', 'sanc', 'wast'])

for dc in df_chn.index:
    if df_chn['chn_moral'][dc] == None:
        chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
    else:
        chn_moral_df.loc[len(chn_moral_df.index)] = list(df_chn['chn_moral'][dc].values())
        
df_chn = pd.concat([df_chn, chn_moral_df], axis=1)
        
df_chn.head()

C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = list(df_chn['chn_moral'][dc].values())
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concat

C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-N

C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
C:\Users\bittersweet\AppData\Local\Temp\ipykernel_144996\2112493011.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-N

,index,商品名称,商品类别,出版年,广告文本,广告标题,chn_moral,altr,auth,care,dili,fair,general,libe,loya,mode,resi,sanc,wast
0,1596,五华牌香烟[May Blossom],烟草制品,1932,兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟 二十枝装每包售国币大洋二角 五十枝装每罐售国币...,"五华牌香烟,""兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟""",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5627,韦廉士红色清导丸[Dr.Willams' Pink Pills For Pale People],药品,1918,讲求卫生为人生本性天理固然也 人生首贵逐日大便通畅有序为天然所当如此也如若大便不利大肠阻塞则...,韦廉士红色清导丸:'讲求卫生为人生本性天理固然也',"{'altr': 0.0, 'auth': 0.25, 'care': 0.25, 'dil...",0.0,0.250000,0.250000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.500000,0.0
2,13532,大炮台香烟[Three Castles Cigarettes],烟草制品,1935,"香味馥郁,不让名花 另有三炮台出售","大炮台香烟,""香味馥郁 不让名花""",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1133,婴孩自己药片[Baby's Own],药品,1930,差肩儿女 秀慧康强 闽有佳音讃羡婴孩自己药片 每年此际小儿患肠胃病者甚多而尤以南方各地天气翳...,"婴孩自己药片,""差肩儿女 秀慧康强 闽有佳音赞美婴孩自己药片""","{'altr': 0.0, 'auth': 0.2, 'care': 0.2, 'dili'...",0.0,0.200000,0.200000,0.0,0.000000,0.0,0.0,0.200000,0.2,0.0,0.200000,0.0
4,3146,亚士北罗药片[Aspro],药品,1933,何以亚士北罗是妇女们的腻友？各国妇女力证亚士北罗药片是她们最需要的药物！为什么？她们的经验知...,"亚士北罗药片,""妇女之腻友""","{'altr': 0.0, 'auth': 0.16666666666666666, 'ca...",0.0,0.166667,0.166667,0.0,0.055556,0.0,0.0,0.388889,0.0,0.0,0.222222,0.0


# 二、情感词典

In [1]:
import pandas as pd
import jieba

**NRC情感词典（NRC Emotion Lexicon）**

In [3]:
#情感词典
nrc = pd.read_excel(r'E:\[- -] C\文本数据分析\上课实例\nrc-emotion-lexicon.xlsx', usecols='A, F, AP:AY')
nrc = nrc.rename(columns={'English Word':'Engword', 'Chinese (simplified) Translation (Google Translate)':'Chnword'})
nrc.head()

D:\anaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Engword,Chnword,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,吓了一跳,0,0,0,0,0,0,0,0,0,0
1,abacus,算盘,0,0,0,0,0,0,0,0,0,1
2,abandon,放弃,0,1,0,0,0,1,0,1,0,0
3,abandoned,弃,0,1,1,0,0,1,0,1,0,0
4,abandonment,放弃,0,1,1,0,0,1,0,1,1,0


## （一）英文语料

In [6]:
#导入语料
df_eng = pd.read_excel(r'E:\[- -] C\文本数据分析\上课实例\text_analysis_twitter_sample.xlsx',index_col = 0)
df_eng.head()

,index,id,screen_name,time,link,text,source
0,49374,890587249372524544,auctnr1,2017-07-27T10:58:41-04:00,https://www.twitter.com/Reuters/statuses/89058...,"RT @Reuters MORE: Top U.S. general says, given...",Twitter for iPhone
1,83246,899354463055618048,SenatorTester,2017-08-20T15:36:27-04:00,https://www.twitter.com/SenatorTester/statuses...,T-minus 2 days until our first-ever Last Best ...,Twitter Web Client
2,100988,903272105738985472,KeithRothfus,2017-08-31T11:03:46-04:00,https://www.twitter.com/KeithRothfus/statuses/...,Please know that help is available. Visit http...,Twitter Web Client
3,193395,921001114409021440,HASCRepublicans,2017-10-19T09:12:31-04:00,https://www.twitter.com/HASCRepublicans/status...,Literally flying the wings off the A-10 in fig...,Twitter Web Client
4,12662,884911451449774080,SteveKnight25,2017-07-11T19:05:05-04:00,https://www.twitter.com/SteveKnight25/statuses...,Today the House unanimously passed my bill #HR...,Twitter Web Client


In [4]:
#设置情感计数公式
Positive, Negative, Anger, Anticipation, Disgust, Fear, Joy, Sadness, Surprise, Trust = [], [], [], [], [], [], [], [], [], []

for idx, row in nrc.iterrows():
    if row['Positive'] == 1:
        Positive.append(row['Engword'])
    if row['Negative'] == 1:
        Negative.append(row['Engword'])
    if row['Anger'] == 1:
        Anger.append(row['Engword'])
    if row['Anticipation'] == 1:
        Anticipation.append(row['Engword'])
    if row['Disgust'] == 1:
        Disgust.append(row['Engword'])
    if row['Fear'] == 1:
        Fear.append(row['Engword'])
    if row['Joy'] == 1:
        Joy.append(row['Engword'])
    if row['Sadness'] == 1:
        Sadness.append(row['Engword'])
    if row['Surprise'] == 1:
        Surprise.append(row['Engword'])
    if row['Trust'] == 1:
        Trust.append(row['Engword'])

In [8]:
#代入语料库计算
emo_nrc_eng = pd.DataFrame(columns=['length_nrc', 'positive_nrc', 'negative_nrc',
                                    'anger_nrc', 'anticipation_nrc', 'disgust_nrc', 'fear_nrc',
                                    'joy_nrc', 'sadness_nrc', 'surprise_nrc', 'trust_nrc'])

for de in df_eng.index:
    positive, negative, anger, anticipation, disgust, fear, joy, sadness, surprise, trust = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    text = df_eng['text'][de].lower()
    wordlist = text.split()
    wordset = set(wordlist)
    wordfreq = []
    
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive += freq
        if word in Negative:
            negative += freq
        if word in Anger:
            anger += freq
        if word in Anticipation:
            anticipation += freq
        if word in Disgust:
            disgust += freq
        if word in Fear:
            fear += freq
        if word in Joy:
            joy += freq
        if word in Sadness:
            sadness += freq
        if word in Surprise:
            surprise += freq
        if word in Trust:
            trust += freq
            
    emotion_info = {
        'length_nrc': len(wordlist),
        'positive_nrc': positive,
        'negative_nrc': negative,
        'anger_nrc': anger,
        'anticipation_nrc': anticipation,
        'disgust_nrc': disgust,
        'fear_nrc': fear,
        'joy_nrc': joy,
        'sadness_nrc': sadness,
        'surprise_nrc': surprise,
        'trust_nrc': trust
    }
    
    emo_info = pd.DataFrame([emotion_info])
    emo_nrc_eng = pd.concat([emo_nrc_eng, emo_info], ignore_index=True)
    
emo_nrc_eng.head()

,length_nrc,positive_nrc,negative_nrc,anger_nrc,anticipation_nrc,disgust_nrc,fear_nrc,joy_nrc,sadness_nrc,surprise_nrc,trust_nrc
0,22,2,0,0,1,0,0,0,0,0,2
1,18,1,0,0,1,0,0,1,0,1,1
2,16,1,0,0,0,0,0,0,0,0,0
3,22,1,1,1,0,0,2,0,0,0,0
4,20,0,1,1,0,1,1,0,1,1,1


In [9]:
#将计算结果合并到df
df_eng = pd.concat([df_eng, emo_nrc_eng], axis=1)
df_eng.head()

,index,id,screen_name,time,link,text,source,length_nrc,positive_nrc,negative_nrc,anger_nrc,anticipation_nrc,disgust_nrc,fear_nrc,joy_nrc,sadness_nrc,surprise_nrc,trust_nrc
0,49374,890587249372524544,auctnr1,2017-07-27T10:58:41-04:00,https://www.twitter.com/Reuters/statuses/89058...,"RT @Reuters MORE: Top U.S. general says, given...",Twitter for iPhone,22,2,0,0,1,0,0,0,0,0,2
1,83246,899354463055618048,SenatorTester,2017-08-20T15:36:27-04:00,https://www.twitter.com/SenatorTester/statuses...,T-minus 2 days until our first-ever Last Best ...,Twitter Web Client,18,1,0,0,1,0,0,1,0,1,1
2,100988,903272105738985472,KeithRothfus,2017-08-31T11:03:46-04:00,https://www.twitter.com/KeithRothfus/statuses/...,Please know that help is available. Visit http...,Twitter Web Client,16,1,0,0,0,0,0,0,0,0,0
3,193395,921001114409021440,HASCRepublicans,2017-10-19T09:12:31-04:00,https://www.twitter.com/HASCRepublicans/status...,Literally flying the wings off the A-10 in fig...,Twitter Web Client,22,1,1,1,0,0,2,0,0,0,0
4,12662,884911451449774080,SteveKnight25,2017-07-11T19:05:05-04:00,https://www.twitter.com/SteveKnight25/statuses...,Today the House unanimously passed my bill #HR...,Twitter Web Client,20,0,1,1,0,1,1,0,1,1,1


## （二）中文语料

In [10]:
df_chn = pd.read_excel(r'E:\[- -] C\文本数据分析\text_analysis_ad.xlsx',index_col = 0)
df_chn.head()

,index,商品名称,商品类别,出版年,广告文本,广告标题
0,1596,五华牌香烟[May Blossom],烟草制品,1932,兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟 二十枝装每包售国币大洋二角 五十枝装每罐售国币...,"五华牌香烟,""兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟"""
1,5627,韦廉士红色清导丸[Dr.Willams' Pink Pills For Pale People],药品,1918,讲求卫生为人生本性天理固然也 人生首贵逐日大便通畅有序为天然所当如此也如若大便不利大肠阻塞则...,韦廉士红色清导丸:'讲求卫生为人生本性天理固然也'
2,13532,大炮台香烟[Three Castles Cigarettes],烟草制品,1935,"香味馥郁,不让名花 另有三炮台出售","大炮台香烟,""香味馥郁 不让名花"""
3,1133,婴孩自己药片[Baby's Own],药品,1930,差肩儿女 秀慧康强 闽有佳音讃羡婴孩自己药片 每年此际小儿患肠胃病者甚多而尤以南方各地天气翳...,"婴孩自己药片,""差肩儿女 秀慧康强 闽有佳音赞美婴孩自己药片"""
4,3146,亚士北罗药片[Aspro],药品,1933,何以亚士北罗是妇女们的腻友？各国妇女力证亚士北罗药片是她们最需要的药物！为什么？她们的经验知...,"亚士北罗药片,""妇女之腻友"""


In [11]:
Positive, Negative, Anger, Anticipation, Disgust, Fear, Joy, Sadness, Surprise, Trust = [], [], [], [], [], [], [], [], [], []

for idx, row in nrc.iterrows():
    if row['Positive'] == 1:
        Positive.append(row['Chnword'])
    if row['Negative'] == 1:
        Negative.append(row['Chnword'])
    if row['Anger'] == 1:
        Anger.append(row['Chnword'])
    if row['Anticipation'] == 1:
        Anticipation.append(row['Chnword'])
    if row['Disgust'] == 1:
        Disgust.append(row['Chnword'])
    if row['Fear'] == 1:
        Fear.append(row['Chnword'])
    if row['Joy'] == 1:
        Joy.append(row['Chnword'])
    if row['Sadness'] == 1:
        Sadness.append(row['Chnword'])
    if row['Surprise'] == 1:
        Surprise.append(row['Chnword'])
    if row['Trust'] == 1:
        Trust.append(row['Chnword'])

In [12]:
#计算语料情感词
emo_nrc_chn = pd.DataFrame(columns=['length_nrc', 'positive_nrc', 'negative_nrc',
                                    'anger_nrc', 'anticipation_nrc', 'disgust_nrc', 'fear_nrc',
                                    'joy_nrc', 'sadness_nrc', 'surprise_nrc', 'trust_nrc'])

for dc in df_chn.index:
    positive, negative, anger, anticipation, disgust, fear, joy, sadness, surprise, trust = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    wordlist = list(jieba.cut(df_chn['广告文本'][dc]))
    wordset = set(wordlist)
    wordfreq = []
    
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive += freq
        if word in Negative:
            negative += freq
        if word in Anger:
            anger += freq
        if word in Anticipation:
            anticipation += freq
        if word in Disgust:
            disgust += freq
        if word in Fear:
            fear += freq
        if word in Joy:
            joy += freq
        if word in Sadness:
            sadness += freq
        if word in Surprise:
            surprise += freq
        if word in Trust:
            trust += freq
            
    emotion_info = {
        'length_nrc': len(wordlist),
        'positive_nrc': positive,
        'negative_nrc': negative,
        'anger_nrc': anger,
        'anticipation_nrc': anticipation,
        'disgust_nrc': disgust,
        'fear_nrc': fear,
        'joy_nrc': joy,
        'sadness_nrc': sadness,
        'surprise_nrc': surprise,
        'trust_nrc': trust
    }
    
    emo_info = pd.DataFrame([emotion_info])
    emo_nrc_chn = pd.concat([emo_nrc_chn, emo_info], ignore_index=True)

emo_nrc_chn.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\BITTER~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built successfully.


,length_nrc,positive_nrc,negative_nrc,anger_nrc,anticipation_nrc,disgust_nrc,fear_nrc,joy_nrc,sadness_nrc,surprise_nrc,trust_nrc
0,30,1,2,0,0,0,0,0,0,0,1
1,129,7,11,1,0,8,2,0,3,0,4
2,11,0,0,0,0,0,0,0,0,0,0
3,155,7,7,2,3,4,3,2,1,0,4
4,433,19,14,4,9,4,4,9,8,4,13


In [13]:
df_chn = pd.concat([df_chn, emo_nrc_chn], axis=1)
df_chn.head()

,index,商品名称,商品类别,出版年,广告文本,广告标题,length_nrc,positive_nrc,negative_nrc,anger_nrc,anticipation_nrc,disgust_nrc,fear_nrc,joy_nrc,sadness_nrc,surprise_nrc,trust_nrc
0,1596,五华牌香烟[May Blossom],烟草制品,1932,兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟 二十枝装每包售国币大洋二角 五十枝装每罐售国币...,"五华牌香烟,""兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟""",30,1,2,0,0,0,0,0,0,0,1
1,5627,韦廉士红色清导丸[Dr.Willams' Pink Pills For Pale People],药品,1918,讲求卫生为人生本性天理固然也 人生首贵逐日大便通畅有序为天然所当如此也如若大便不利大肠阻塞则...,韦廉士红色清导丸:'讲求卫生为人生本性天理固然也',129,7,11,1,0,8,2,0,3,0,4
2,13532,大炮台香烟[Three Castles Cigarettes],烟草制品,1935,"香味馥郁,不让名花 另有三炮台出售","大炮台香烟,""香味馥郁 不让名花""",11,0,0,0,0,0,0,0,0,0,0
3,1133,婴孩自己药片[Baby's Own],药品,1930,差肩儿女 秀慧康强 闽有佳音讃羡婴孩自己药片 每年此际小儿患肠胃病者甚多而尤以南方各地天气翳...,"婴孩自己药片,""差肩儿女 秀慧康强 闽有佳音赞美婴孩自己药片""",155,7,7,2,3,4,3,2,1,0,4
4,3146,亚士北罗药片[Aspro],药品,1933,何以亚士北罗是妇女们的腻友？各国妇女力证亚士北罗药片是她们最需要的药物！为什么？她们的经验知...,"亚士北罗药片,""妇女之腻友""",433,19,14,4,9,4,4,9,8,4,13


**大连理工大学情感词典（DLUT Emotion Lexicon）**

In [14]:
#语料库阅览
dlut = pd.read_excel(r'E:\[- -] C\文本数据分析\上课实例\dlut-emotion-lexicon.xlsx', usecols=['词语', '词性种类', '情感分类', '强度'])
dlut.head()

,词语,词性种类,情感分类,强度
0,脏乱,adj,NN,7
1,糟报,adj,NN,5
2,早衰,adj,NE,5
3,责备,verb,NN,5
4,贼眼,noun,NN,5


In [15]:
# 整理情感词典
Happy, Good, Surprise, Anger, Sad, Fear, Disgust  = [], [], [], [], [], [], []

for idx, row in dlut.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])     
    if row['情感分类'] in ['NA']:
        Anger.append(row['词语'])    
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust

In [16]:
#计算情感词
emo_dlut = pd.DataFrame(columns=['length_dlut', 'positive_dlut', 'negative_dlut',
                                'anger_dlut', 'disgust_dlut', 'fear_dlut', 'good_dlut',
                                'sadness_dlut', 'surprise_dlut', 'happy_dlut'])

for dc in df_chn.index:
    positive, negative, anger, disgust, fear, sad, surprise, good, happy = 0, 0, 0, 0, 0, 0, 0, 0, 0
    wordlist = list(jieba.cut(df_chn['广告文本'][dc]))
    wordset = set(wordlist)
    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive += freq
        if word in Negative:
            negative += freq
        if word in Anger:
            anger += freq
        if word in Disgust:
            disgust += freq
        if word in Fear:
            fear += freq
        if word in Sad:
            sad += freq
        if word in Surprise:
            surprise += freq
        if word in Good:
            good += freq
        if word in Happy:
            happy += freq
            
    emotion_info = {
        'length_dlut': len(wordlist),
        'positive_dlut': positive,
        'negative_dlut': negative,
        'anger_dlut': anger,
        'disgust_dlut': disgust,
        'fear_dlut': fear,
        'good_dlut': good,
        'sadness_dlut': sad,
        'surprise_dlut': surprise,
        'happy_dlut': happy
    }
    
    emo_info = pd.DataFrame([emotion_info])
    emo_dlut = pd.concat([emo_dlut, emo_info], ignore_index=True)
    
emo_dlut.head()

,length_dlut,positive_dlut,negative_dlut,anger_dlut,disgust_dlut,fear_dlut,good_dlut,sadness_dlut,surprise_dlut,happy_dlut
0,30,0,0,0,0,0,0,0,0,0
1,129,9,6,0,3,2,9,1,0,0
2,11,0,0,0,0,0,0,0,0,0
3,155,7,6,0,5,0,7,1,0,0
4,433,34,16,0,14,1,23,1,0,11


In [17]:
#合并df，可视化计算结果
df_chn = pd.concat([df_chn, emo_dlut], axis=1)
df_chn.head()

,index,商品名称,商品类别,出版年,广告文本,广告标题,length_nrc,positive_nrc,negative_nrc,anger_nrc,...,length_dlut,positive_dlut,negative_dlut,anger_dlut,disgust_dlut,fear_dlut,good_dlut,sadness_dlut,surprise_dlut,happy_dlut
0,1596,五华牌香烟[May Blossom],烟草制品,1932,兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟 二十枝装每包售国币大洋二角 五十枝装每罐售国币...,"五华牌香烟,""兰勃脱白脱勒公司 五华牌香烟 佛及尼埃香烟""",30,1,2,0,...,30,0,0,0,0,0,0,0,0,0
1,5627,韦廉士红色清导丸[Dr.Willams' Pink Pills For Pale People],药品,1918,讲求卫生为人生本性天理固然也 人生首贵逐日大便通畅有序为天然所当如此也如若大便不利大肠阻塞则...,韦廉士红色清导丸:'讲求卫生为人生本性天理固然也',129,7,11,1,...,129,9,6,0,3,2,9,1,0,0
2,13532,大炮台香烟[Three Castles Cigarettes],烟草制品,1935,"香味馥郁,不让名花 另有三炮台出售","大炮台香烟,""香味馥郁 不让名花""",11,0,0,0,...,11,0,0,0,0,0,0,0,0,0
3,1133,婴孩自己药片[Baby's Own],药品,1930,差肩儿女 秀慧康强 闽有佳音讃羡婴孩自己药片 每年此际小儿患肠胃病者甚多而尤以南方各地天气翳...,"婴孩自己药片,""差肩儿女 秀慧康强 闽有佳音赞美婴孩自己药片""",155,7,7,2,...,155,7,6,0,5,0,7,1,0,0
4,3146,亚士北罗药片[Aspro],药品,1933,何以亚士北罗是妇女们的腻友？各国妇女力证亚士北罗药片是她们最需要的药物！为什么？她们的经验知...,"亚士北罗药片,""妇女之腻友""",433,19,14,4,...,433,34,16,0,14,1,23,1,0,11
